### **(KR)**

- 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
- 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
- 항상 세 개의 이모티콘으로 답장하도록 `FewShotPromptTemplate` 또는 `FewShotChatMessagePromptTemplate`을 사용하여 체인에 예시를 제공하세요.
- 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.

In [6]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "hi"}, {"output": "hello"})
memory.save_context({"input": "hi"}, {"output": "hello"})
memory.save_context({"input": "hi"}, {"output": "hello"})
memory.save_context({"input": "hi"}, {"output": "hello"})
memory.save_context({"input": "hi"}, {"output": "hello"})

print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='hi'), AIMessage(content='hello'), HumanMessage(content='hi'), AIMessage(content='hello'), HumanMessage(content='hi'), AIMessage(content='hello'), HumanMessage(content='hi'), AIMessage(content='hello'), HumanMessage(content='hi'), AIMessage(content='hello')]}


In [7]:
from langchain.memory import ConversationBufferWindowMemory


memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
)


def add_message(input, output):
    memory.save_context({"input": {input}}, {"output": {output}})


add_message(1, 1)
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)

print(memory.load_memory_variables({}))

{'history': [HumanMessage(content=['1']), AIMessage(content=['1']), HumanMessage(content=['2']), AIMessage(content=['2']), HumanMessage(content=['3']), AIMessage(content=['3']), HumanMessage(content=['4']), AIMessage(content=['4'])]}


In [8]:
add_message(5, 5)
add_message(6, 6)
print(memory.load_memory_variables({}))

{'history': [HumanMessage(content=['3']), AIMessage(content=['3']), HumanMessage(content=['4']), AIMessage(content=['4']), HumanMessage(content=['5']), AIMessage(content=['5']), HumanMessage(content=['6']), AIMessage(content=['6'])]}


이제부터 LLM 사용

summary - 요약해줌


In [10]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": {input}}, {"output": {output}})


def get_history():
    return memory.load_memory_variables({})


add_message("hi, i'm lution. i live in bupyung", "wow that is so cool!")

In [11]:
add_message("bupyung is so pretty", "i wisg i could go!!!")

In [14]:
print(get_history())

{'history': "Lution introduces themselves as living in Bupyung, and the AI responds enthusiastically, saying it's cool. Lution mentions how pretty Bupyung is, and the AI expresses a wish to go there."}


In [18]:
get_history()

{'history': "Lution introduces themselves as living in Bupyung, and the AI responds enthusiastically, saying it's cool. Lution mentions how pretty Bupyung is, and the AI expresses a wish to go there."}

이번엔 summary + buffer

오래된 애들은 요약함


In [21]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=10,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": {input}}, {"output": {output}})


def get_history():
    return print(memory.load_memory_variables({}))


add_message("hi, i'm lution. i live in bupyung", "wow that is so cool!")

In [25]:
add_message("hi, i'm lution. i live in bupyung", "wow that is so cool!")
add_message("i like a meat", "wow that is so cool!")
add_message("I like a IU who is famous singer.", "wow she is so great singer!")
add_message("i do crossfit workout everyday.", "wow you are so nice!")

In [27]:
add_message("i like playing piano.", "wow nice work!")

In [28]:
get_history()

{'history': 'System: Lution introduces themselves as living in Bupyung, and the AI responds enthusiastically, saying it\'s cool that Lution lives there. The human mentions liking IU, a famous singer, and the AI agrees, saying she is a great singer. The human shares that they do crossfit workouts everyday, to which the AI responds with admiration, calling them nice. The human then mentions liking playing piano, to which the AI responds with a compliment, saying "wow nice work!"'}


가장 중요한 요약본 뽑는거

대화 중 엔티티의 knowledge graph 만들기

knowledget graph에서 히스토리를 가지고 오지 않고 엔티티를 가져오기 때문에 get_history 삭제



In [29]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": {input}}, {"output": {output}})


add_message("hi, i'm lution. i live in bupyung", "wow that is so cool!")

In [30]:
memory.load_memory_variables({"input": "who is lution?"})

{'history': [SystemMessage(content='On lution: lution lives in Bupyung.')]}

In [31]:
add_message("lution likes movies of marvle's", "wow that is so awesome!")

In [32]:
memory.load_memory_variables({"input": "what does lution likes?"})

{'history': [SystemMessage(content="On lution: lution lives in Bupyung. lution likes movies of marvle's.")]}

# LCEL 에서 memory 사용하는법
- memory를 chain 에 꽂는 방법

- 두 종류의 chain을 사용해서 꽂는 방법

LLM Chain: off-the-shelf chain -> 일반적인 체인

커스텀 Chain: LCEL 을 활용해서 만들수있음.





In [37]:
# off-the-shelf chain 사용기 - LLMChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True,
)

# 예는 predict? -
chain.predict(question="my name is nico")



> Entering new LLMChain chain...
Prompt after formatting:
my name is nico

> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [38]:
chain.predict(question="I live in bupyung.")



> Entering new LLMChain chain...
Prompt after formatting:
I live in bupyung.

> Finished chain.


"That's great! Bupyung is a district in Incheon, South Korea. What do you enjoy most about living in Bupyung?"

In [35]:
chain.predict(question="what is my name?")

"I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information."

ConversationSummaryBufferMemory 을 사용한 LLM 체인은 이전 데이터를 저장하지 못함.

디버깅 해보자 - verbose=True 사용

LLMChain 은 이전에 한 말을 기억하지 못하지만 memory 는 기억한다.

그렇다면 문제는 memory 가 프롬프트에 포함되지 않는다는 것.

해결방법은 프롬프트에 포함시키자



In [39]:
# off-the-shelf chain 사용기 - LLMChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=120, memory_key="chat_history"
)

# template 추가
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human: {question}
    You: 
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

# 예는 predict? -
chain.predict(question="my name is nico")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human: my name is nico
    You: 


> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [46]:
chain.predict(question="my name is nico")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    System: Nico introduces themselves to the AI, who greets Nico and offers assistance. Nico mentions they live in Bupyung, and the AI acknowledges this and offers help with information or services related to Bupyung. The human asks the AI for their name, and the AI responds that their name is Nico. The human mentions they live in Bupyung, and the AI offers assistance with information or services related to Bupyung.
Human: I live in bupyung.
AI: It seems like you mentioned that you live in Bupyung multiple times. Is there something specific you would like to know or discuss about Bupyung?
Human: my name is nico
AI: Hello Nico! How can I assist you today?
Human: I live in bupyung.
AI: That's great to know, Nico! How can I assist you with information or services related to Bupyung?
    Human: my name is nico
    You: 


> Finished chain.


'Hello Nico! How can I assist you today?'

In [47]:
chain.predict(question="I live in bupyung.")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    System: Nico introduces themselves to the AI, who greets Nico and offers assistance. Nico mentions they live in Bupyung, and the AI acknowledges this and offers help with information or services related to Bupyung. The human asks the AI for their name, and the AI responds that their name is Nico. The human mentions they live in Bupyung, and the AI offers assistance with information or services related to Bupyung. The human reiterates that they live in Bupyung, prompting the AI to inquire if there is something specific they would like to know or discuss about Bupyung.
Human: my name is nico
AI: Hello Nico! How can I assist you today?
Human: I live in bupyung.
AI: That's great to know, Nico! How can I assist you with information or services related to Bupyung?
Human: my name is nico
AI: Hello Nico! How can I assist you today?
    Human: I live in bupyung.
    You: 


> Finished 

"That's great to know, Nico! How can I assist you with information or services related to Bupyung? Is there something specific you would like to know or discuss about Bupyung?"

In [41]:
chain.predict(question="what is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: my name is nico
AI: Nice to meet you, Nico! How can I assist you today?
Human: I live in bupyung.
AI: That's great to know! How can I assist you with information or services related to Bupyung?
    Human: what is my name?
    You: 


> Finished chain.


'Your name is Nico.'

In [42]:
chain.predict(question="I live in bupyung.")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: my name is nico
AI: Nice to meet you, Nico! How can I assist you today?
Human: I live in bupyung.
AI: That's great to know! How can I assist you with information or services related to Bupyung?
Human: what is my name?
AI: Your name is Nico.
    Human: I live in bupyung.
    You: 


> Finished chain.


"That's great to know! How can I assist you with information or services related to Bupyung?"

# 대화형 Memory 사용
- ChatPromptTemplate 사용
- MessagesPlaceholder : 대화형 메세지를 담을 공간


In [48]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful AI talking to a human."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

# 예는 predict? -
chain.predict(question="my name is nico")



> Entering new LLMChain chain...
Prompt after formatting:
System: you are a helpful AI talking to a human.
Human: my name is nico

> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

# LCEL 로 커스텀 chain 에 memory 추가하기
- chain.invoke 사용
- RunnablePassthrough.assign() 사용 
  : 프롬프트가 format 되기 전에 우리가 함수를 실행시키는 걸 허락해줌.
   그리고 이 함수에서 우리가 원하는 어떤 값이든 변수에 할당 가능.
   그 변수는 prompt 로 전달됨.




In [49]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful AI talking a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = prompt | llm

In [50]:
# chain.invoke() - case 1
chain.invoke(
    {
        "chat_history": memory.load_memory_variables({})["chat_history"],
        "question": "My name is lution",
    }
)
# 이 접근방식의 문제는 체인을 호출할 때마다 chat_history도 추가해 주어야 한다는 것.

AIMessage(content='Hello Lution! How can I assist you today?')

In [52]:
from langchain.schema.runnable import RunnablePassthrough


def load_memory(input):
    """RunnablePassthrough.assing 은 사용자 input을 받는다."""
    return memory.load_memory_variables({})["chat_history"]


chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [53]:
invoke_chain("my name is lution")

content='Hello Lution! How can I assist you today?'


In [54]:
invoke_chain("what is my name?")

content='Your name is Lution.'
